# Phase 1: Environment & Setup
Building a production RAG system, started in a notebook to **prototype components** before moving them to Python modules.

### Step 1: Load Environment Variables

In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Adjust 'RAG systems using LlamaIndex' if your folder name is different
env_path = Path("..") / "RAG systems using LlamaIndex" / ".env"

# 2. Load it
if env_path.exists():
    load_dotenv(dotenv_path=env_path)
    print("✅ Loaded .env file")
else:
    print(f"❌ Could not find .env at: {env_path.resolve()}")

# 3. Verify
if os.getenv("GOOGLE_API_KEY"):
    print("✅ API Key found")
else:
    print("⚠️ API Key not found. Check your .env file.")

✅ Loaded .env file
✅ API Key found


### Step 2: The Ingestion Logic
**Design Decision**: Clone **Everything**, Filter **Later**.
*   **Cloning**: Use `git clone`. It's faster and simpler to just pull the whole repo than to try and "sparse checkout" file-by-file.
*   **Filtering**: We will apply a **Blacklist** (exclude `.png`, `.pth`, `.git`) during the document loading phase. This keeps the local disk faithful to reality but protects the embedding model from garbage.

In [3]:
import subprocess
import shutil

# CONFIG
REPO_URL = "https://github.com/XunpengYi/Diff-Retinex-Plus"
STORAGE_PATH = Path("./github_rag_engine/data/repos")
REPO_NAME = REPO_URL.split("/")[-1]
LOCAL_REPO_PATH = STORAGE_PATH / REPO_NAME

def clone_repo(url, path):
    if path.exists():
        print(f"ℹ️ Repo already exists at: {path}")
        # In production, we would git pull here
        return
    
    print(f"⏳ Cloning {url}...")
    try:
        subprocess.run(["git", "clone", url, str(path)], check=True)
        print("✅ Clone successful")
    except subprocess.CalledProcessError as e:
        print(f"❌ Clone failed: {e}")

# 1. Run the clone
clone_repo(REPO_URL, LOCAL_REPO_PATH)

ℹ️ Repo already exists at: github_rag_engine/data/repos/Diff-Retinex-Plus


### Step 3: Filtering & Processing
Now we iterate through the files. 

**The Rule**: skip if extension is in `IGNORED_EXTENSIONS` OR if it is a hidden file/directory (starts with `.`).

In [4]:
IGNORED_EXTENSIONS = {
    # Images
    ".png", ".jpg", ".jpeg", ".gif", ".svg", ".ico",
    # Compiled / Binary
    ".pyc", ".o", ".exe", ".dll", ".so", ".dylib",
    # Models / Data
    ".pth", ".pt", ".onnx", ".pkl", ".bin", ".data",
    # Git
    ".git",
    # Lock files (Optional - debating this)
    ".lock"
}


In [5]:

def get_source_files(directory):
    source_files = []
    for root, dirs, files in os.walk(directory):
        # 1. Remove hidden dirs like .git in-place to avoid traversing them
        dirs[:] = [d for d in dirs if not d.startswith(".")]
        
        for file in files:
            # 2. Skip hidden files
            if file.startswith("."):
                continue
                
            # 3. Check extension blacklist
            ext = os.path.splitext(file)[1].lower()
            if ext in IGNORED_EXTENSIONS:
                continue
            
            full_path = Path(root) / file
            source_files.append(full_path)
            
    return source_files

In [6]:

# Test the filter
files = get_source_files(LOCAL_REPO_PATH)
print(f"Found {len(files)} valid source files")
print("Sample:", [f.name for f in files[:5]])

Found 44 valid source files
Sample: ['test.py', 'README.md', 'test_UHD.py', 'train.py', 'environment.yaml']


### Step 4: The "Repo Map" (The "What was here" context)
You made a great point: if we skip `.pth` files, the LLM won't know they exist. 

**The Solution**: We create a single special document called the **Repo Map**. 
This is a text file containing the tree structure of the *entire* repo (including the files we ignored). We index this first. This gives the application "Spatial Awareness".

In [7]:
def generate_repo_map(root_dir):
    lines = []
    root_path = Path(root_dir)

    for root, dirs, files in os.walk(root_path):
        if ".git" in dirs:
            dirs.remove(".git")

        for f in files:
            rel_path = Path(root, f).relative_to(root_path)
            ext = rel_path.suffix.lower()

            marker = ""
            if f.startswith(".") or ext in IGNORED_EXTENSIONS:
                marker = " [IGN]"

            lines.append(f"{rel_path.as_posix()}{marker}")

    return "\n".join(sorted(lines))

In [8]:
# Generate and Print
repo_map_str = generate_repo_map(LOCAL_REPO_PATH)
print("--- REPO MAP PREVIEW ---")
print(repo_map_str)

--- REPO MAP PREVIEW ---
README.md
archs/DRNet_Retinex_arch.py
archs/MoE_arch.py
archs/__init__.py
archs/ddpm_arch.py
archs/decom_arch.py
archs/sr3unet_arch.py
asserts/framework.png [IGN]
data/__init__.py
data/lol_dataset.py
dataset/put_the_dataset_here.md
decom_tools/decom_arch.py
decom_tools/loss_decom_TDN.py
decom_tools/my_dataset.py
decom_tools/test_decom_checkpoint.py
decom_tools/train_decom.py
decom_tools/transforms.py
decom_tools/utils.py
decom_weights/put_the_decom_weights_here.md
environment.yaml
lolv1_offical_results/1.png [IGN]
lolv1_offical_results/111.png [IGN]
lolv1_offical_results/146.png [IGN]
lolv1_offical_results/179.png [IGN]
lolv1_offical_results/22.png [IGN]
lolv1_offical_results/23.png [IGN]
lolv1_offical_results/493.png [IGN]
lolv1_offical_results/547.png [IGN]
lolv1_offical_results/55.png [IGN]
lolv1_offical_results/665.png [IGN]
lolv1_offical_results/669.png [IGN]
lolv1_offical_results/748.png [IGN]
lolv1_offical_results/778.png [IGN]
lolv1_offical_results/780.

### Step 5: Loading with Metadata
Now we load the *content* of the valid files. 

**Crucial**: We inject `file_metadata`. Every chunk must know its own filename.

In [ ]:
from llama_index.core import SimpleDirectoryReader, Document

In [ ]:
# 1. Create the Repo Map Document
map_doc = Document(
    text=repo_map_str,
    metadata={"file_path": "REPO_STRUCTURE.txt", "type": "structure"})

In [ ]:
import datetime
# 2. To extract metadata from files
def file_metadata_extractor(file_path):
    # Get modification time
    #stat = os.stat(file_path)
    #timestamp = stat.st_mtime
    # Format as readable string
    #last_mod = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")

    return {
        "file_path": file_path,
        "file_name": os.path.basename(file_path),
        "extension": os.path.splitext(file_path)[1],
        "repo_path": os.path.dirname(file_path),
        #"last_modified": last_mod
    }

In [12]:
if not files:
    print("⚠️ No files found to index!")
    documents = []
else:
    reader = SimpleDirectoryReader(
        input_files=[str(f) for f in files],
        file_metadata=file_metadata_extractor
    )
    code_documents = reader.load_data()
    all_documents = [map_doc] + code_documents
    print(f"✅ Loaded {len(code_documents)} code documents + 1 Repo Map")

✅ Loaded 44 code documents + 1 Repo Map


In [13]:
print(code_documents[0].metadata)

{'file_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus/test.py', 'file_name': 'test.py', 'extension': '.py', 'repo_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus'}


### Step 6: Settings (OpenAI)
We use **OpenAI** for both embedding and LLM.

In [14]:
# Imports
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

In [15]:
def configure_llm_settings():
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
    print("✅ Configured: text-embedding-3-small & GPT-4o-mini")

In [16]:
configure_llm_settings()

✅ Configured: text-embedding-3-small & GPT-4o-mini


### Step 7: Adaptive Chunking (File-Type Aware & Tiktoken)
You are absolutely right. `.py` is handled by `CodeSplitter`, but `.md` needs `MarkdownNodeParser`, and `.yaml` needs standard text splitting.

**Refinement**: We use **tiktoken** to ensure our chunks are measured in *tokens*, not characters. This is critical for maximizing the LLM's context window usage.

In [17]:
import tiktoken
from llama_index.core.node_parser import CodeSplitter, MarkdownNodeParser, SentenceSplitter

In [ ]:
def get_nodes_adaptive(docs):
    alln = []
    tok = tiktoken.encoding_for_model("gpt-4o-mini").encode

    ps = CodeSplitter(
        language="python",
        chunk_lines=150,#125
        chunk_lines_overlap=30,
        max_chars=3000#2500
    )

    ms = MarkdownNodeParser(
        chunk_size=800,
        chunk_overlap=100
    )

    ts = SentenceSplitter(
        chunk_size=512,
        chunk_overlap=50,
        tokenizer=tok
    )

    for doc in docs:
        fn = doc.metadata.get("file_name", "")
        ext = doc.metadata.get("extension", "").lower()

        if ext == ".py":
            nds = ps.get_nodes_from_documents([doc])
        elif ext == ".md":
            nds = ms.get_nodes_from_documents([doc])
        else:
            nds = ts.get_nodes_from_documents([doc])

        for i, n in enumerate(nds):
            n.metadata["chunk_id"] = i
            n.metadata["file_ext"] = ext
            n.metadata["file_name"] = fn

            pre = nds[i - 1].text if i > 0 else ""
            post = nds[i + 1].text if i < len(nds) - 1 else ""

            n.metadata["pre_context"] = pre
            n.metadata["post_context"] = post

        alln.extend(nds)

    return alln

In [19]:
def inspect_nodes(nodes, sample_exts=(".py", ".md", ".yaml", ".yml")):
    seen = set()
    for n in nodes:
        ext = n.metadata.get("file_ext", "")
        if ext in sample_exts and ext not in seen:
            seen.add(ext)
            print("\n--- Sample Chunk for", ext, n.metadata.get("file_name", ""))
            print("[Node Type: {}]".format(n.class_name()))
            print("chunk_id:", n.metadata.get("chunk_id"))
            txt = n.text[:400].replace("\n", "\\n")
            print(txt + "...")
            pre = n.metadata.get("pre_context", "")
            post = n.metadata.get("post_context", "")
            print("\npre_context (first 200 chars):")
            print(pre[:200].replace("\n", "\\n"))
            print("\npost_context (first 200 chars):")
            print(post[:200].replace("\n", "\\n"))



In [ ]:
# run
nodes = get_nodes_adaptive(code_documents)
print("generated nodes:", len(nodes))

generated nodes: 170

--- Sample Chunk for .py test.py
[Node Type: TextNode]
chunk_id: 0
import logging\nimport torch\nfrom os import path as osp\nimport archs\nimport data\nimport losses\nimport models\n\nfrom basicsr.data import build_dataloader, build_dataset\nfrom basicsr.models import build_model\nfrom basicsr.utils import get_env_info, get_root_logger, get_time_str, make_exp_dirs\nfrom basicsr.utils.options import dict2str, parse_options\n\n\ndef test_pipeline(root_path):\n    # parse options,...

pre_context (first 200 chars):


post_context (first 200 chars):


--- Sample Chunk for .md README.md
[Node Type: TextNode]
chunk_id: 0
# [TPAMI 2025] Diff-Retinex++: Retinex-Driven Reinforced Diffusion Model for Low-Light Image Enhancement...

pre_context (first 200 chars):


post_context (first 200 chars):
### [Paper](https://ieeexplore.ieee.org/abstract/document/10974676) | [Code](https://github.com/XunpengYi/Diff-Retinex-Plus) \n\n**Diff-Retinex++: Retinex-Driven Reinforced Diffusio

In [ ]:
inspect_nodes(nodes)
empty_exts = [n for n in nodes if not n.metadata.get("file_ext")]
print("nodes with empty file_ext:", len(empty_exts))
if empty_exts:
    for n in empty_exts[:5]:
        print("file_name:", n.metadata.get("file_name"), "chunk_id:", n.metadata.get("chunk_id"))

In [21]:
nodes[99].metadata

{'file_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus/scripts/options.py',
 'file_name': 'options.py',
 'extension': '.py',
 'repo_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus/scripts',
 'chunk_id': 3,
 'file_ext': '.py',
 'pre_context': 'parser = argparse.ArgumentParser()\n    parser.add_argument(\'-opt\', type=str, default="options/train_diff_retinex_plus_lolv1.yml", help=\'Path to option YAML file.\')\n    parser.add_argument(\'--launcher\', choices=[\'none\', \'pytorch\', \'slurm\'], default=\'none\', help=\'job launcher\')\n    parser.add_argument(\'--auto_resume\', action=\'store_true\')\n    parser.add_argument(\'--debug\', action=\'store_true\')\n    parser.add_argument(\'--local_rank\', type=int, default=0)\n    parser.add_argument(\n        \'--force_yml\', nargs=\'+\', default=None, help=\'Force to update yml files. Examples: train:ema_decay=0.999\')\n    args = parser.parse_args()\n\n    # parse yml to dict\n    with open(args.opt, mode=\'r\') as f:\n     

# Step 8: Embeddings with Caching

In [ ]:

from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.core.storage.kvstore import SimpleKVStore

In [23]:
# 1. Setup the Cache Storage
CACHE_FILE = "./github_rag_engine/cache/ingestion_cache.json"

In [24]:
os.makedirs(os.path.dirname(CACHE_FILE), exist_ok=True)
if os.path.exists(CACHE_FILE):
    print(f"📖 Loading ingestion cache from {CACHE_FILE}...")
    kv_store = SimpleKVStore.from_persist_path(CACHE_FILE)
else:
    print("✨ Creating new ingestion cache...")
    kv_store = SimpleKVStore()

📖 Loading ingestion cache from ./github_rag_engine/cache/ingestion_cache.json...


In [25]:
# 2. Define the Pipeline
pipeline = IngestionPipeline(
    transformations=[
        Settings.embed_model  # The embedding model is configured earlier
    ],
    cache=IngestionCache(cache=kv_store)
)

In [26]:
print(f"🚀 Generating embeddings for {len(nodes)} nodes (with caching)...")
nodes_with_embeddings = pipeline.run(nodes=nodes)
kv_store.persist(CACHE_FILE)
print(f"✅ Embeddings generated and cache saved to {CACHE_FILE}")

🚀 Generating embeddings for 170 nodes (with caching)...
✅ Embeddings generated and cache saved to ./github_rag_engine/cache/ingestion_cache.json


In [27]:
print(f"Sample Embedding (First 5 dims): {nodes_with_embeddings[0].embedding[:5]}")

Sample Embedding (First 5 dims): [-0.014153656549751759, -0.018069013953208923, 0.06104068085551262, -0.055009569972753525, -0.01625724881887436]


In [29]:
len(nodes_with_embeddings[0].embedding)

1536

In [30]:
import tiktoken

In [37]:
def calculate_token_counts(nodes, model_name="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model_name)
    total_tokens = 0
    counts = []
    print(f"📊 Analyzing {len(nodes)} nodes with tiktoken ({model_name})...")
    for i, node in enumerate(nodes):
        # method .get_content() ensures we get the text formatted as the LLM sees it
        tokens = encoding.encode(node.get_content())
        count = len(tokens)
        # 2. Inject into metadata (Useful for debugging!)
        #node.metadata["token_count"] = count
        
        counts.append(count)
        total_tokens += count
        # Print a few samples
        if i%20: 
            print(f"   [Node {i}] {node.metadata.get('file_name')}: {count} tokens")

    # 3. Summary Stats
    avg_tokens = sum(counts) / len(counts) if counts else 0
    max_tokens = max(counts) if counts else 0
    
    print(f"\n--- Token Stats ---")
    print(f"✅ Total Tokens: {total_tokens:,}")
    print(f"   Average:      {avg_tokens:.1f}")
    print(f"   Max:          {max_tokens}")
    print(f"   Min:          {min(counts) if counts else 0}")
    
    # Cost Estimate (gpt-4o-mini input is ~$0.15 per 1M tokens)
    cost = (total_tokens / 1_000_000) * 0.15
    print(f"   Est. Cost:    ${cost:.6f}")

In [38]:
calculate_token_counts(nodes)

📊 Analyzing 170 nodes with tiktoken (gpt-4o-mini)...
   [Node 1] README.md: 29 tokens
   [Node 2] README.md: 98 tokens
   [Node 3] README.md: 57 tokens
   [Node 4] README.md: 204 tokens
   [Node 5] README.md: 480 tokens
   [Node 6] README.md: 294 tokens
   [Node 7] README.md: 436 tokens
   [Node 8] README.md: 185 tokens
   [Node 9] test_UHD.py: 454 tokens
   [Node 10] train.py: 80 tokens
   [Node 11] environment.yaml: 466 tokens
   [Node 12] environment.yaml: 458 tokens
   [Node 13] environment.yaml: 471 tokens
   [Node 14] environment.yaml: 469 tokens
   [Node 15] environment.yaml: 464 tokens
   [Node 16] environment.yaml: 468 tokens
   [Node 17] environment.yaml: 158 tokens
   [Node 18] train_pipline.py: 320 tokens
   [Node 19] train_pipline.py: 451 tokens
   [Node 21] train_pipline.py: 19 tokens
   [Node 22] train_pipline.py: 661 tokens
   [Node 23] train_pipline.py: 517 tokens
   [Node 24] train_pipline.py: 36 tokens
   [Node 25] transforms.py: 758 tokens
   [Node 26] transforms.py

%pip install chromadb llama-index-vector-stores-chroma

# Step 9: Vector Store Setup (ChromaDB)

In [41]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex

In [43]:
# 1. Initialize ChromaDB (Persistent)
db_path = "./github_rag_engine/data/chroma_db"
print(f"📂 Opening database at {db_path}...")

chroma_client = chromadb.PersistentClient(path=db_path)
# 2. Create Collection
# A collection is like a table. We name it specifically for this repo.
collection = chroma_client.get_or_create_collection("github_rag_v1")

📂 Opening database at ./github_rag_engine/data/chroma_db...


In [44]:
# 3. Connect LlamaIndex to Chroma
vector_store = ChromaVectorStore(chroma_collection=collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# 4. Build the Index (Persist to Disk)
# We pass 'nodes_with_embeddings' so it doesn't re-calculate costs!
print("💾 Saving index to disk... (This might take a moment to write to SQLite)")

index = VectorStoreIndex(
    nodes_with_embeddings,
    storage_context=storage_context
)
print(f"Index saved! Collection now has {collection.count()} chunks.")

💾 Saving index to disk... (This might take a moment to write to SQLite)
Index saved! Collection now has 170 chunks.


# Step 10: Retrieval & Quality Control

In [45]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [71]:
# 1. The Retriever
# We fetch slightly more (k=10) than we need, so we can filter them down later.
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)
# 2. Post-Processor: "Quality Control"
# This acts as a gatekeeper. Nodes below the cutoff are dropped.
# Note: For OpenAI text-embedding-3, scores are often lower, so we start with 0.4.
similarity_cutoff = SimilarityPostprocessor(similarity_cutoff=0.25)

In [72]:
# 3. The Query Engine
# Retriever (Fetch 10) -> PostProcessor (Filter bad ones) -> LLM (Synthesize Answer)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[similarity_cutoff]
)

print(f"✅ Query Engine Ready (Top-10 Retrieval -> Cutoff 0.25)")

✅ Query Engine Ready (Top-10 Retrieval -> Cutoff 0.25)


In [ ]:
test_query = "Where is the main training code implemented?"
print(f"\n🔎 Inspecting retrieval for: '{test_query}'")

retrieved_nodes = retriever.retrieve(test_query)

for i, node in enumerate(retrieved_nodes):
    # Check if it would pass our cutoff
    status = "✅ KEEP" if node.score >= 0.25 else "❌ DROP"
    print(f"[{i}] {status} (Score: {node.score:.4f}) File: {node.metadata.get('file_name')}")


🔎 Inspecting retrieval for: 'Where is the main training code implemented?'


2026-02-04 15:30:48,880 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[0] ✅ KEEP (Score: 0.3118) File: train_pipline.py
[1] ✅ KEEP (Score: 0.3034) File: train_pipline.py
[2] ✅ KEEP (Score: 0.2961) File: train_pipline.py
[3] ✅ KEEP (Score: 0.2897) File: train_pipline.py
[4] ✅ KEEP (Score: 0.2717) File: train_decom.py
[5] ✅ KEEP (Score: 0.2675) File: train_decom.py
[6] ✅ KEEP (Score: 0.2633) File: train.py
[7] ✅ KEEP (Score: 0.2630) File: train_diff_retinex_plus_lolv2_real.yml
[8] ✅ KEEP (Score: 0.2621) File: train_decom.py
[9] ✅ KEEP (Score: 0.2610) File: README.md


# Step 11: Custom Prompt Engineering

In [74]:
from llama_index.core import PromptTemplate

In [75]:
qa_prompt_str = (
    "You are a Senior Software Engineer acting as a code assistant.\n"
    "You have been provided with context from a GitHub repository.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given this context, answer the following technical question.\n"
    "Rules:\n"
    "1. Use ONLY the context provided. If the answer is not in the code, say 'I cannot find the answer in the provided files'.\n"
    "2. When citing code, mention the specific filename (e.g., 'in train.py...').\n"
    "3. Be concise but technical.\n"
    "\n"
    "Question: {query_str}\n"
    "Answer:"
)

In [76]:
text_qa_template = PromptTemplate(qa_prompt_str)
# 3. Update the Query Engine
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": text_qa_template}
)

print(f"✅ Custom 'Senior Engineer' prompt applied.")

✅ Custom 'Senior Engineer' prompt applied.


In [77]:
# --- TEST WITH CUSTOM PROMPT ---
query_str = "What is the main model, in which file is it present and what is the overall goal of the model?"
print(f"\n❓ Querying: '{query_str}'")

response = query_engine.query(query_str)
print("\n--- 🤖 Expert Response ---")
print(response)


❓ Querying: 'What is the main model, in which file is it present and what is the overall goal of the model?'


2026-02-04 15:30:51,797 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 15:30:54,223 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



--- 🤖 Expert Response ---
The main model is `DRNetModel`, which is present in the file `github_rag_engine/data/repos/Diff-Retinex-Plus/models/DRNet_model.py`. The overall goal of the model is to perform image restoration, specifically enhancing images by utilizing a combination of networks including a denoising function and a decomposition network to improve the quality of low-quality images (lq) and generate high-quality outputs (gt). The model incorporates various loss functions to optimize its performance during training.



# Step 12: Automated Evaluation (LLM-as-a-Judge)

In [78]:
from llama_index.core.evaluation import FaithfulnessEvaluator, RelevancyEvaluator
import pandas as pd

In [79]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=Settings.llm)
relevancy_evaluator = RelevancyEvaluator(llm=Settings.llm)

def evaluate_query(query_str):
    print(f"🧪 Evaluating Query: '{query_str}'")
    response = query_engine.query(query_str)
    pass_faithfulness = faithfulness_evaluator.evaluate_response(response=response)
    pass_relevancy = relevancy_evaluator.evaluate_response(query=query_str, response=response)
    
    # 3. Report Results
    print(f"   Shape of Response: {len(str(response))} chars")
    print(f"   Sources Retrieved: {len(response.source_nodes)}")
    
    print("\n--- 👩‍⚖️ Verdict ---")
    print(f"   Faithfulness (No Hallucinations): {'✅ PASS' if pass_faithfulness.passing else '❌ FAIL'}")
    print(f"   Relevancy    (Answered Question): {'✅ PASS' if pass_relevancy.passing else '❌ FAIL'}")
    
    if not pass_faithfulness.passing:
        print(f"   ⚠️ Reason: {pass_faithfulness.feedback}")
    if not pass_relevancy.passing:
        print(f"   ⚠️ Reason: {pass_relevancy.feedback}")
        
    return response

In [ ]:
eval_query_str = "How are the losses calculated in this repo? detailed check"
response = evaluate_query(eval_query_str)

print("\n--- Actual Answer Content ---")
print(response)

🧪 Evaluating Query: 'How are the losses calculated in this repo? detailed check'


2026-02-04 15:32:33,918 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 15:32:55,209 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 15:32:59,718 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 15:32:59,795 - INFO - Retrying request to /chat/completions in 0.476411 seconds
2026-02-04 15:33:01,596 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


   Shape of Response: 1601 chars
   Sources Retrieved: 10

--- 👩‍⚖️ Verdict ---
   Faithfulness (No Hallucinations): ✅ PASS
   Relevancy    (Answered Question): ✅ PASS

--- Actual Answer Content ---
The losses in this repository are calculated primarily in the `Decom_Loss` class found in `loss_decom_TDN.py`. The loss computation involves several components:

1. **Reconstruction Losses**:
   - `self.recon_loss_low`: Calculated using L1 loss between the product of `R_low` and `L_low_3` and `I_low`.
   - `self.recon_loss_high`: Calculated using L1 loss between the product of `R_high` and `L_high_3` and `I_high`.
   - `self.recon_loss_crs_low`: Calculated using L1 loss between the product of `R_high` and `L_low_3` and `I_low`.
   - `self.recon_loss_crs_high`: Calculated using L1 loss between the product of `R_low` and `L_high_3` and `I_high`.

2. **Equalization Loss**:
   - `self.equal_R_loss`: Calculated using L1 loss between `R_low` and the detached `R_high`.

3. **Smoothness Losses**:
 

In [81]:
eval_query_str = "Do we need any predefined .pth file to run this model, We yes what to do for new dataset"
response = evaluate_query(eval_query_str)

print("\n--- Actual Answer Content ---")
print(response)

🧪 Evaluating Query: 'Do we need any predefined .pth file to run this model, We yes what to do for new dataset'


2026-02-04 15:35:14,941 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 15:35:20,005 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 15:35:20,078 - INFO - Retrying request to /chat/completions in 0.405129 seconds
2026-02-04 15:35:21,353 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 15:35:21,397 - INFO - Retrying request to /chat/completions in 0.436064 seconds
2026-02-04 15:35:22,431 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


   Shape of Response: 374 chars
   Sources Retrieved: 10

--- 👩‍⚖️ Verdict ---
   Faithfulness (No Hallucinations): ✅ PASS
   Relevancy    (Answered Question): ✅ PASS

--- Actual Answer Content ---
Yes, you need predefined .pth files (pretrained weights) to run the model. For a new dataset, you should download the pretrained weights and place them in the `pretrained_weights` folder as mentioned in the README.md file. The specific links for downloading the pretrained weights for various datasets are provided in the README.md under the section "3. Pretrained Weights".
